# NLP PROJECT

Import

In [ ]:
from nltk.translate.metrics import *
import string
import numpy as np
from nltk import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
#nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from sklearn.pipeline import Pipeline

Creating the dataframe 

In [ ]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t', names=['label', 'message'])

In [ ]:
for i in range(len(messages)):
     messages['message'][i] = messages['message'][i].lower()

In [ ]:
stopwords.words('english')

Use of stopwords

In [ ]:
 def rem_stopwords(text): 
    
  #  nopunc = [char for char in text if char not in string.punctuation]
# nopunc = ''.join(nopunc)
    
    stop_words = stopwords.words("english") + ['ok']
    word_tokens = word_tokenize(text) 
    
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words] 
    
    return filtered_text 

In [ ]:
def text_process(mess):

    #Supprime les ponctuations
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
     # stopwords
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [ ]:
messages['message'].apply(rem_stopwords).head(6)

In [ ]:
messages['message'].apply(text_process).head(6)

Vectorisation

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
boW = CountVectorizer(analyzer=rem_stopwords).fit(messages["message"])

In [ ]:
#Sparse matrix 
message_bow = boW.transform(messages['message'])

In [ ]:
print('Sparse matrix shape: ', message_bow.shape)

TF-IDF

In [ ]:
#Import 
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# Create weighted matrix
boW = CountVectorizer(analyzer=rem_stopwords).fit(messages["message"])
#Sparse matrix 
message_bow = boW.transform(messages['message'])

In [ ]:
#TF IDF
tfidf_trans = TfidfTransformer().fit(message_bow)
mess_tfidf = tfidf_trans.transform(message_bow)

In [ ]:
#Import
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# model 
model_spam = MultinomialNB().fit(mess_tfidf,messages['label'])

# Modelisation

In [ ]:
from sklearn.model_selection  import train_test_split

In [ ]:
# training et test set 
mess_train,mess_test,label_train,label_test = train_test_split(messages['message'], messages['label'], test_size=0.4)

In [ ]:
# pipeline pour eviter la redondance

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=rem_stopwords)),  # creation des tokens
    ('tfidf', TfidfTransformer()),  # calcul des poids tfifd 
    ('classifier', MultinomialNB()),  # utilisation du model
])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=rem_stopwords)),  # creation des tokens
    ('tfidf', TfidfTransformer()),  # calcul des poids tfifd 
    ('classifier', RandomForestClassifier()),  # utilisation du model
])


In [ ]:
pipeline.fit(mess_train,label_train) # training du model

In [ ]:
prediction = pipeline.predict(mess_test)

Mesure de performance

In [ ]:
from sklearn.metrics import *
from nltk.metrics.scores import *

In [ ]:
def performance(y_test,y_model):
    print ("accuracy", accuracy_score(y_test,y_model))
    print ("classification_report",classification_report(y_test,y_model))
    print ("F1 score",f1_score(y_test,y_model, average='macro'))
    #print ("AUC ROC",roc_auc_score(y_test,y_model))

In [ ]:
accuracy_score(label_test,prediction)

In [ ]:
performance(label_test,prediction)

In [ ]:
performance(label_test,prediction)

# GRIDSEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

param = { 
    'classifier__n_estimators' : [2,5,10,20,50],
    'classifier__max_features' : ['auto','sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8,None],
    'classifier__criterion' :['gini', 'entropy'],
   # 'classifier__min_samples_split': [2, 5, 10, 15,20],
   # 'classifier__min_samples_leaf': [1, 2, 5, 10]   

}

grid = GridSearchCV(pipeline, cv=5, n_jobs=1, param_grid=param  ,scoring='accuracy')

In [ ]:
from sklearn.linear_model import LogisticRegression
pipe = Pipeline([("classifier", RandomForestClassifier())])

grid_param = [
              #  {"classifier": [LogisticRegression()]
              #   },
                {
                 "classifier__n_estimators": [10, 100, 1000],
                 "classifier__max_depth":[5,8,15,25,30,None],
                 "classifier__min_samples_leaf":[1,2,5,10,15,100],
                 "classifier__max_leaf_nodes": [2, 5,10]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipeline, grid_param, cv=5, verbose=0,n_jobs=-1) # Fit

In [ ]:
grid.fit(label_test,prediction) 
grid.best_params_

In [ ]:
grid.estimator.get_params().keys()

In [ ]:
best_grid = grid.best_estimator_
#rid_accuracy = evaluate(best_grid,label_test,prediction)
best_grid

In [ ]:
prediction_SV = best_grid.predict(mess_test)
performance(label_test,prediction)